# BNN指标

数据集 MNIST

推断方法 BBB

In [1]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))

import BayesKeras
from BayesKeras import PosteriorModel
from BayesKeras import analyzers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np

D:\Users\T470\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'BBB', 'gpu': '1'}
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob']
# the variational inference method
opt = dict['opt']
inference = opt

## accuracy

### 查看模型

In [15]:
"""
model = Sequential()
model.add(Dense(512, activation="relu", input_shape=(None, None, 28 * 28)))
model.add(Dense(10, activation="softmax"))
"""
model = PosteriorModel("%s_FCN_Posterior_%s_epochs20_dnnLayer1" % (inference, rob))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 


==========================posterior_mean_start==========================
[<tf.Tensor: shape=(784, 512), dtype=float32, numpy=
 array([[-1.03877785e-04, -3.28878959e-04,  3.05326976e-05, ...,
         -1.75361220e-05,  1.90327191e-04,  1.16797673e-04],
        [ 1.68560320e-0

### 输入

In [16]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_test = X_test / 255.
# X_test = X_test.astype("float32").reshape(-1, 28*28)

In [17]:
X_test[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [18]:
y_test[0]

7

In [19]:
from tqdm import trange

loss = tf.keras.losses.SparseCategoricalCrossentropy()

num_images = 500

### 输出

In [20]:
accuracy = tf.keras.metrics.Accuracy()
preds = model.predict(X_test[0:500])

In [21]:
preds[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([6.4575563e-16, 9.0342628e-10, 1.4820283e-06, 3.0794646e-07,
       9.6148838e-13, 5.6874452e-09, 4.1274480e-15, 9.9999827e-01,
       4.5951524e-12, 1.9768773e-08], dtype=float32)>

In [22]:
y_pred = np.argmax(preds, axis=1)  # 最大元素的索引值
y_pred

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 3, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 7, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 9, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 9, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 4, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 5, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 8,
       8, 7, 2, 3, 0, 6, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 0, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 4, 2, 9, 2, 0, 4, 0, 0, 2, 8, 6,

In [23]:
y_true = y_test[0:500]
y_true

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [24]:
accuracy.update_state(y_pred, y_true)
print("%s Accuracy: " % (inference), accuracy.result())

BBB Accuracy:  tf.Tensor(0.944, shape=(), dtype=float32)


In [25]:
accuracy = tf.keras.metrics.Accuracy()
preds = model.predict(X_train)
y_pred = np.argmax(preds, axis=1)  # 最大元素的索引值
y_true = y_train
accuracy.update_state(y_pred, y_true)
print("%s Accuracy: " % (inference), accuracy.result())

BBB Accuracy:  tf.Tensor(0.9285, shape=(), dtype=float32)


## robustness

鲁棒率：对输入数据加入扰动后的准确率

In [ ]:
accuracy = tf.keras.metrics.Accuracy()
# 用PGD去近似输入点最坏情况的扰动
adv = analyzers.PGD(model, X_test[0:500], eps=0.1, loss_fn=loss, num_models=10)

In [ ]:
adv[0]

In [ ]:
preds = model.predict(adv)

In [26]:
y_pred = np.argmax(preds, axis=1)
y_pred

array([9, 6, 2, 2, 9, 7, 9, 8, 4, 9, 5, 5, 7, 5, 3, 3, 3, 3, 3, 5, 7, 5,
       1, 4, 9, 0, 2, 9, 0, 8, 5, 8, 5, 5, 9, 5, 2, 3, 5, 8, 7, 7, 9, 3,
       5, 3, 2, 4, 9, 9, 2, 7, 4, 4, 6, 5, 7, 2, 7, 8, 2, 2, 9, 2, 4, 9,
       0, 9, 2, 2, 7, 5, 0, 7, 8, 9, 2, 7, 8, 9, 9, 2, 5, 9, 4, 4, 3, 6,
       5, 8, 3, 6, 8, 5, 8, 9, 8, 8, 5, 8, 8, 5, 7, 9, 9, 7, 8, 8, 4, 9,
       2, 8, 8, 7, 9, 4, 9, 6, 4, 5, 3, 4, 9, 2, 7, 4, 3, 8, 3, 3, 6, 6,
       3, 2, 5, 8, 5, 8, 2, 4, 2, 2, 2, 3, 4, 8, 5, 3, 5, 9, 2, 5, 4, 8,
       7, 5, 5, 5, 0, 4, 8, 2, 5, 7, 4, 3, 9, 1, 8, 7, 2, 3, 2, 3, 8, 3,
       3, 9, 3, 2, 3, 2, 9, 5, 2, 4, 3, 3, 5, 8, 1, 4, 2, 9, 5, 3, 8, 8,
       9, 8, 2, 8, 8, 3, 2, 5, 4, 3, 6, 4, 4, 5, 4, 7, 4, 5, 2, 6, 5, 3,
       7, 0, 8, 9, 1, 5, 4, 9, 2, 9, 5, 1, 6, 7, 9, 9, 3, 1, 9, 3, 8, 8,
       0, 4, 2, 2, 0, 2, 9, 2, 4, 8, 4, 4, 3, 9, 2, 2, 6, 6, 5, 8, 2, 9,
       7, 3, 0, 3, 5, 5, 5, 5, 3, 8, 3, 7, 3, 9, 8, 7, 1, 8, 3, 7, 0, 6,
       2, 9, 8, 6, 8, 5, 7, 6, 5, 9, 8, 5, 6, 3, 6,

In [27]:
y_true = y_test[0:500]
y_true

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2,
       3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4,
       6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3,
       6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4,
       8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6,
       5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5,
       1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7,
       1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6,
       4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5,
       7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9,
       8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7,
       9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2,
       6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4,

In [28]:
accuracy.update_state(y_pred, y_true)
fgsm = accuracy.result()
print("FGSM Robustness: ", accuracy.result())

FGSM Robustness:  tf.Tensor(0.148, shape=(), dtype=float32)
